In [2]:
import keras
from keras_cv.layers import DropPath
from keras import ops
from tensorflow.keras import *
from tensorflow.keras.layers import *
import tensorflow as tf  
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.initializers import RandomNormal
from functools import partial
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2024-05-09 10:10:35.662603: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-09 10:10:35.662717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-09 10:10:35.771599: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class PatchEmbed(layers.Layer):
    """Patch embedding block.

    Args:
        embed_dim: feature size dimension.
    """

    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = (img_size, img_size)  
        patch_size = (patch_size, patch_size)  
        num_patches = (img_size[0] // patch_size[0]) * (img_size[1] // patch_size[1])
        self.patch_shape = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches
        
#         applying filters to the input images
# 
        self.proj = Conv2D(filters=embed_dim,
                                           kernel_size=patch_size,
                                           strides=patch_size,
                                           padding='valid')

    def call(self, inputs, **kwargs):
        B, H, W, C = inputs.shape # batch size, height,width,channel
        assert (H, W) == self.img_size, f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        
#         convert the input image patches to embdeddings
        x = self.proj(inputs)
        x = tf.reshape(x, [B, self.num_patches, -1])  # reshpate to [batch_size, num_patch, embed_dim]
        x = tf.transpose(x, perm=[0, 2, 1])  # Transpose to match PyTorch's output shape

        return x

In [4]:
def get_sinusoid_encoding_table(n_position, d_hid): 
    ''' Sinusoid position encoding table ''' 

    def get_position_angle_vec(position): 
        return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)] 

    sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)]) 
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2]) # dim 2i 
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2]) # dim 2i+1 
    
    
#     convert the np_array to float32
    tf_sinusoid_table = tf.convert_to_tensor(sinusoid_table, dtype=tf.float32)
#     add a new dimension
    tf_sinusoid_table = tf.expand_dims(tf_sinusoid_table, axis=0)
    
    
    return tf_sinusoid_table

In [5]:
def _cfg(url='', **kwargs):
    return {
        'url': url,
        'num_classes': 1000, 'input_size': (3, 224, 224), 'pool_size': None,
        'crop_pct': .9, 'interpolation': 'bicubic',
        'mean': (0.5, 0.5, 0.5), 'std': (0.5, 0.5, 0.5),
        **kwargs
    }

In [6]:
class Mlp(layers.Layer):
    def __init__(self, in_features, hidden_features=None, out_features=None, 
                 act_layer=tf.nn.gelu, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        
        self.fc1 = layers.Dense(hidden_features)
        self.activation = act_layer
        self.fc2 = layers.Dense(out_features)
        self.drop = layers.Dropout(drop)  # layers.Dropout for dropout

    def call(self, x, training=None):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.drop(x, training=training)  #  self.drop for dropout
        return x

In [7]:
class Attention(layers.Layer):
    
    def __init__(self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0., 
                 proj_drop=0., attn_head_dim=None):
        super().__init__()
        
        self.num_heads = num_heads
        head_dim = dim // num_heads
        if attn_head_dim is not None:
            head_dim = attn_head_dim
        all_head_dim = head_dim * self.num_heads
        self.scale = qk_scale or head_dim ** -0.5
        
        self.qkv = layers.Dense(all_head_dim * 3, use_bias=False)
        
        if qkv_bias:
            self.q_bias = self.add_weight(shape=(all_head_dim,), initializer="zeros", trainable=True)
            self.v_bias = self.add_weight(shape=(all_head_dim,), initializer="zeros", trainable=True)
        else:
            self.q_bias = None
            self.v_bias = None
            
        self.attn_drop = layers.Dropout(attn_drop)
        self.proj = layers.Dense(units=dim, input_dim=all_head_dim)
        self.proj_drop = layers.Dropout(proj_drop)

    def call(self, x, training=None):
        B, N, C = tf.shape(x)[0], tf.shape(x)[1], tf.shape(x)[2]
        qkv_bias = None
        
        if self.q_bias is not None:
            qkv_bias = tf.concat([self.q_bias, tf.zeros_like(self.v_bias)], axis=0)

        qkv = tf.matmul(x, self.qkv.weight)
        qkv = tf.nn.bias_add(qkv, qkv_bias)
        qkv = tf.reshape(qkv, (B, N, 3, self.num_heads, -1))
        qkv = tf.transpose(qkv, perm=[2, 0, 3, 1, 4])
        q, k, v = qkv[0], qkv[1], qkv[2]

        q = q * self.scale
        attn = tf.matmul(q, tf.transpose(k, perm=[0, 1, 3, 2]))

        attn = tf.nn.softmax(attn, axis=-1)
        attn = self.attn_drop(attn, training=training)

        x = tf.matmul(attn, v)
        x = tf.transpose(x, perm=[0, 2, 1, 3])
        x = tf.reshape(x, (B, N, -1))

        x = self.proj(x)
        x = self.proj_drop(x, training=training)

        return x

In [8]:
class DropPath(layers.Layer):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)
    
    def extra_repr(self) -> str:
        return 'p={}'.format(self.drop_prob)

In [9]:
class Block(layers.Layer):
#     def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
#                  drop_path=0., init_values=None, act_layer=tf.nn.gelu, norm_layer=LayerNormalization, attn_head_dim=None):
    def __init__(self, dim, num_heads, mlp_ratio, qkv_bias, qk_scale, drop, attn_drop, drop_path, init_values, act_layer, norm_layer, attn_head_dim):

        super().__init__()
        
#         self.norm1 = norm_layer(epsilon=1e-6)
#         self.attn = Attention(
#             dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
#             attn_drop=attn_drop, proj_drop=drop, attn_head_dim=attn_head_dim)
        
#         self.drop_path = tf.keras.layers.Dropout(rate=drop_path)
#         self.norm2 = norm_layer(epsilon=1e-6)
#         mlp_hidden_dim = int(dim * mlp_ratio)
#         self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)
#         if init_values is not None:
#             self.gamma_1 = self.add_weight(shape=(dim,), initializer=tf.constant_initializer(init_values), trainable=True)
#             self.gamma_2 = self.add_weight(shape=(dim,), initializer=tf.constant_initializer(init_values), trainable=True)
#         else:
#             self.gamma_1, self.gamma_2 = None, None
        self.norm1 = norm_layer(epsilon=1e-6)
        self.attn = Attention(
            use_scale=qk_scale,
            dropout=attn_drop,
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            head_dim=attn_head_dim,  # Example attention head dimension
            name="attention"
        )
        self.drop_path = tf.keras.layers.Dropout(rate=drop_path)
        self.norm2 = norm_layer(epsilon=1e-6)

    def call(self, inputs, **kwargs):
        x = inputs
        x = self.norm1(x)
        x = self.attn(x)
        x = self.drop_path(x)
        x = self.norm2(x)
        return x

In [10]:
class PretrainVisionTransformerEncoder(layers.Layer):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=0, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., norm_layer=LayerNormalization, init_values=None,
                 use_learnable_pos_emb=False):
        super().__init__()
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim
        
        #         patch embedding

        self.patch_embed = PatchEmbed(img_size=img_size, patch_size=patch_size, in_chans=in_chans,
                                      embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches
        
        # positional encoding       

        if use_learnable_pos_emb:
            self.pos_embed = tf.Variable(tf.zeros((1, num_patches + 1, embed_dim)))
        else:
            self.pos_embed = get_sinusoid_encoding_table(num_patches, embed_dim)
            
        #             Transformer blocks


        # convert drop_path_rate to a float
        drop_path_rate = float(drop_path_rate)

        dpr = tf.linspace(0.0, drop_path_rate, depth)  # create a linear space from 0.0 to drop_path_rate

        # convert the TensorFlow tensor to a list of Python floats
        dpr = dpr.numpy().tolist()


        self.blocks = [Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, 
                qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], 
                norm_layer=LayerNormalization,
                init_values=init_values)
                       for i in range(depth)]
        
        # classification
#         self.norm = norm_layer(epsilon=1e-6, axis=-1)
        self.norm = norm_layer(epsilon=1e-6)
        self.head = Dense(num_classes, input_shape=(embed_dim,)) if num_classes > 0 else tf.identity
        
        # Learnable positional embeddings
        if use_learnable_pos_emb:
            self.pos_embed = self.add_weight(
                "pos_embed", shape=(1, embed_dim), initializer=trunc_normal_(stddev=0.02)
            )

    def reset_classifier(self, num_classes):
        self.num_classes = num_classes
        self.head = Dense(num_classes, kernel_initializer=RandomNormal(stddev=0.02))

    def call(self, x, mask):
        x = self.patch_embed(x)
        x = x + self.pos_embed
        for blk in self.blocks:
            x = blk(x)
        x = self.norm(x)
        x = self.head(x)
        return x

In [11]:
class PretrainVisionTransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, patch_size=16, num_classes=768, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., 
                 attn_drop_rate=0.,
                 drop_path_rate=0., norm_layer=LayerNormalization, init_values=None,num_patches=196,):
        super().__init__()
        self.num_classes = num_classes
        assert num_classes == 3 * patch_size ** 2
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models
        self.patch_size = patch_size
        
        drop_path_rate = float(drop_path_rate)
        # drop probabilities using TensorFlow and numpy
        dpr = np.linspace(0, drop_path_rate, depth).tolist()

        # convert the list of drop probabilities to TensorFlow tensors
        dpr = [tf.convert_to_tensor(value=prob, dtype=tf.float32) for prob in dpr]
        self.blocks = [Block(dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, 
                             qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], 
                norm_layer=LayerNormalization,
                init_values=init_values)
                       for i in range(depth)]
        
#         self.norm = norm_layer(epsilon=1e-6, axis=-1)
        self.norm = norm_layer(epsilon=1e-6)
        self.head = Dense(num_classes, input_shape=(embed_dim,)) if num_classes > 0 else tf.identity


    def reset_classifier(self, num_classes):
        self.num_classes = num_classes
        self.head = Dense(num_classes, kernel_initializer=GlorotUniform())

    def call(self, x, return_token_num):
        for blk in self.blocks:
            x = blk(x)

        if return_token_num > 0:
            x = self.head(self.norm(x[:, -return_token_num:, :]))  # Return only the mask tokens to predict pixels
        else:
            x = self.head(self.norm(x))  # [B, N, 3*16^2]

        return x

In [12]:
class PretrainVisionTransformer(tf.keras.Model):
    def __init__(self,
                 img_size=224, 
                 patch_size=16, 
                 encoder_in_chans=3, 
                 encoder_num_classes=0, 
                 encoder_embed_dim=768, 
                 encoder_depth=12,
                 encoder_num_heads=12, 
                 decoder_num_classes=768, 
                 decoder_embed_dim=512, 
                 decoder_depth=8,
                 decoder_num_heads=8, 
                 mlp_ratio=4., 
                 qkv_bias=False, 
                 qk_scale=None, 
                 drop_rate=0., 
                 attn_drop_rate=0.,
                 drop_path_rate=0., 
                 norm_layer=LayerNormalization, 
                 init_values=0.,
                 use_learnable_pos_emb=False,
                 num_classes=0, # avoid the error from create_fn in timm
                 in_chans=0, # avoid the error from create_fn in timm
                 ):
        super().__init__()

        self.encoder = PretrainVisionTransformerEncoder(
            img_size=img_size, 
            patch_size=patch_size, 
            in_chans=encoder_in_chans, 
            num_classes=encoder_num_classes, 
            embed_dim=encoder_embed_dim, 
            depth=encoder_depth,
            num_heads=encoder_num_heads, 
            mlp_ratio=mlp_ratio, 
            qkv_bias=qkv_bias, 
            qk_scale=qk_scale, 
            drop_rate=drop_rate, 
            attn_drop_rate=attn_drop_rate,
            drop_path_rate=drop_path_rate, 
            norm_layer=norm_layer, 
            init_values=init_values,
            use_learnable_pos_emb=use_learnable_pos_emb)

        self.decoder = PretrainVisionTransformerDecoder(
            patch_size=patch_size, 
            num_patches=self.encoder.patch_embed.num_patches,
            num_classes=decoder_num_classes, 
            embed_dim=decoder_embed_dim, 
            depth=decoder_depth,
            num_heads=decoder_num_heads, 
            mlp_ratio=mlp_ratio, 
            qkv_bias=qkv_bias, 
            qk_scale=qk_scale, 
            drop_rate=drop_rate, 
            attn_drop_rate=attn_drop_rate,
            drop_path_rate=drop_path_rate, 
            norm_layer=norm_layer, 
            init_values=init_values)

        self.encoder_to_decoder = Dense(decoder_embed_dim, activation=None, use_bias=False)

        self.mask_token = tf.Variable(tf.zeros((1, 1, decoder_embed_dim)))

        num_patches = self.encoder.patch_embed.num_patches
        self.pos_embed = get_sinusoid_encoding_table(self.encoder.patch_embed.num_patches, decoder_embed_dim)

        # Initialize the mask token using truncated normal distribution
        self.mask_token.assign(tf.random.truncated_normal(self.mask_token.shape, stddev=0.02))

    def call(self, x, mask):
        x_vis = self.encoder(x, mask)  # [B, N_vis, C_e]
        x_vis = self.encoder_to_decoder(x_vis)  # [B, N_vis, C_d]

        B, N, C = x_vis.shape
        
        # Prepare positional embeddings based on mask
        expand_pos_embed = tf.tile(self.pos_embed, [B, 1, 1])
        pos_emd_vis = tf.boolean_mask(expand_pos_embed, ~mask, axis=1)  # Visible positional embeddings
        pos_emd_mask = tf.boolean_mask(expand_pos_embed, mask, axis=1)  # Masked positional embeddings
        
        # Concatenate input features with positional embeddings
        x_full = tf.concat([x_vis + pos_emd_vis, self.mask_token + pos_emd_mask], axis=1)
        
        # Decode the masked token predictions
        x = self.decoder(x_full, pos_emd_mask.shape[1])  # [B, N_mask, 3 * 16 * 16]
        return x

# Helper function to initialize weights
def _init_weights(m):
    if isinstance(m, Dense):
        GlorotUniform()(m.weights)
        if m.bias is not None:
            Zeros()(m.bias)
    elif isinstance(m, LayerNormalization):
        Zeros()(m.bias)
        tf.ones_like(m.weights)

In [13]:
pre_transformer = PretrainVisionTransformer()

TypeError: Block.__init__() missing 2 required positional arguments: 'act_layer' and 'attn_head_dim'

In [ ]:
def pretrain_mae_base_patch16_224(pretrained=False, **kwargs):
    # Define the model architecture parameters
    model = PretrainVisionTransformer(
        img_size=224,
        patch_size=16, 
        encoder_embed_dim=768, 
        encoder_depth=12, 
        encoder_num_heads=12,
        encoder_num_classes=0,
        decoder_num_classes=768,
        decoder_embed_dim=384,
        decoder_depth=4,
        decoder_num_heads=6,
        mlp_ratio=4, 
        qkv_bias=True,
        norm_layer=partial(LayerNormalization(epsilon=1e-6)),
        **kwargs)

    return model

In [ ]:
model = pretrain_mae_base_patch16_224()

In [ ]:
model.summary()

### Data Augmentation

In [ ]:
class RandomMaskingGenerator:
    def __init__(self, input_size, mask_ratio):
        self.input_size = input_size
        self.mask_ratio = mask_ratio

    def __call__(self):
        num_patches = self.input_size[0] * self.input_size[1]
        num_mask = int(self.mask_ratio * num_patches)
        mask = np.hstack([np.zeros(num_patches - num_mask), np.ones(num_mask)])
        np.random.shuffle(mask)
        return mask.astype(int)

def build_pretraining_dataset(args):

    image_data_generator = ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=False,
        rescale=1/255,
    )

    def custom_generator(image_paths):
        for img_path in image_paths:
            img = Image.open(img_path)  
            img_array = np.array(img)
            augmented_img = image_data_generator.random_transform(img_array)
            masked_positions = RandomMaskingGenerator(augmented_img.shape[:2], args.mask_ratio)()
            yield augmented_img, masked_positions

    return custom_generator(args.data_path)